In [1]:
import pandas as pd
import numpy as np

In [108]:
train = pd.read_csv('./data/gro_train.csv').iloc[:, 1:]
valid = pd.read_csv('./data/gro_test.csv').iloc[:, 1:]

In [109]:
valid

,0,1,2
0,0.364810,0.341317,0.328201
1,0.309653,0.161677,0.324055
2,0.340982,0.209581,0.327135
3,0.352785,0.263473,0.317660
4,0.138555,0.269461,0.317660
...,...,...,...
195,0.140099,0.143713,0.267914
196,0.179812,0.167665,0.253464
197,0.105019,0.113772,0.253464
198,0.265527,0.179641,0.253464


In [110]:
train

,0,1,2
0,0.000000,0.000000,0.792965
1,0.292554,0.000000,0.792965
2,0.233977,0.000000,0.790951
3,0.226806,0.000000,0.792728
4,0.244457,0.000000,0.793675
...,...,...,...
1509,0.335245,0.185629,0.319673
1510,0.307998,0.173653,0.313159
1511,0.149366,0.137725,0.313159
1512,0.307998,0.161677,0.313159


In [111]:
train.iloc[1:1+1, :].values

array([[0.29255378, 0.        , 0.79296459]])

In [112]:
valid

,0,1,2
0,0.364810,0.341317,0.328201
1,0.309653,0.161677,0.324055
2,0.340982,0.209581,0.327135
3,0.352785,0.263473,0.317660
4,0.138555,0.269461,0.317660
...,...,...,...
195,0.140099,0.143713,0.267914
196,0.179812,0.167665,0.253464
197,0.105019,0.113772,0.253464
198,0.265527,0.179641,0.253464


In [113]:
time_steps = 100
ts_train_len = len(train)
ts_valid_len = len(valid)
output = 3
for_periods = 14

In [114]:
X_train = []
y_train = []
for i in range(time_steps, ts_train_len-1):
    X_train.append(train.iloc[i-time_steps:i])
    y_train.append(train.iloc[i:i+1, :].values.squeeze().tolist())
X_train, y_train = np.array(X_train), np.array(y_train)

In [115]:
X_valid = []
y_valid = []
for i in range(time_steps, ts_valid_len-1):
    X_valid.append(valid.iloc[i-time_steps:i])
    y_valid.append(valid.iloc[i:i+1].values.squeeze().tolist())
X_valid, y_valid = np.array(X_valid), np.array(y_valid)

In [116]:
X_train.shape, y_train.shape

((1413, 100, 3), (1413, 3))

In [117]:
X_valid.shape, y_valid.shape

((99, 100, 3), (99, 3))

In [129]:
from tensorflow import keras

model = keras.models.Sequential([
        keras.layers.LSTM(50, input_shape=[None, X_train.shape[2]], return_sequences=True, activation='tanh'),
        keras.layers.LSTM(50, activation='tanh'),
        keras.layers.Dense(output)
])

In [130]:
model.compile(loss=keras.losses.MeanSquaredError(), optimizer=keras.optimizers.Adam())

In [ ]:
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid))

Epoch 1/100
45/45 [==============================] - 6s 85ms/step - loss: 0.0199 - val_loss: 0.0049
Epoch 2/100
45/45 [==============================] - 3s 71ms/step - loss: 0.0037 - val_loss: 0.0046
Epoch 3/100
45/45 [==============================] - 3s 72ms/step - loss: 0.0036 - val_loss: 0.0046
Epoch 4/100
45/45 [==============================] - 3s 71ms/step - loss: 0.0036 - val_loss: 0.0044
Epoch 5/100
45/45 [==============================] - 3s 70ms/step - loss: 0.0035 - val_loss: 0.0045
Epoch 6/100
45/45 [==============================] - 3s 72ms/step - loss: 0.0034 - val_loss: 0.0043
Epoch 7/100
45/45 [==============================] - 3s 73ms/step - loss: 0.0034 - val_loss: 0.0044
Epoch 8/100
45/45 [==============================] - 3s 75ms/step - loss: 0.0034 - val_loss: 0.0042
Epoch 9/100
45/45 [==============================] - 3s 76ms/step - loss: 0.0034 - val_loss: 0.0048
Epoch 10/100
45/45 [==============================] - 3s 76ms/step - loss: 0.0034 - val_loss: 0.0042

In [ ]:
import matplotlib.pyplot as plt
y_vloss = history.history['val_loss']
y_loss = history.history['loss']

x_len = np.arange(len(y_loss))
plt.plot(x_len, y_vloss, marker='.', c='red', label="Validation-set Loss")
plt.plot(x_len, y_loss, marker='.', c='blue', label="Train-set Loss")
plt.title(f'time_steps : {time_steps}, periods : {for_periods}')
plt.legend(loc='upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()


### 한개 예측

In [ ]:
 model.predict(X_valid)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (10, 4))
plt.plot(valid.iloc[:-1, 0])
plt.plot(range(1+time_steps, len(X_valid) + time_steps+1), model.predict(X_valid)[:, 0])
plt.vlines(time_steps+1, 0, 1, color = 'red', linestyle = '--')
plt.ylim(valid.iloc[:-1, 0].values.min() - 0.1,valid.iloc[:-1, 0].values.max()+0.1)
plt.legend(['True', 'Predict'])
plt.show()

### 여러개 예측

In [ ]:
def period_prediction(model, X, for_periods):
    predict = []
    X_pred = X.copy()
    for _ in range(for_periods):
        pred = model.predict(np.expand_dims(X_pred, axis = 0))
        predict += pred.tolist()
        X_pred = np.append(X_pred[1:], model.predict(np.expand_dims(X_pred,axis = 0)), axis = 0)
    
    return np.array(predict)

In [ ]:
i = 0

predict = period_prediction(model, X_valid[i], for_periods)

plt.figure(figsize= (15, 4))

# period
plt.plot(np.arange(time_steps + for_periods), valid.iloc[i:time_steps + for_periods+i, 0].values)

# True
plt.plot(np.arange(time_steps, time_steps + for_periods), valid.iloc[i+time_steps:time_steps + for_periods+i, 0].values)

# predict
plt.plot(np.arange(time_steps, time_steps + for_periods) , predict[:, 0])

plt.vlines(time_steps, 0, 1, color = 'gray', linestyle = '--')
plt.vlines(time_steps + 7, 0, 1, color = 'red', linestyle = '--')
plt.ylim(valid.iloc[:time_steps + for_periods, 0].values.min() - 0.1,valid.iloc[:time_steps + for_periods, 0].values.max() + 0.2)
plt.title(f', time_steps : {time_steps}, periods : {for_periods}')
plt.grid()
plt.legend(['period', 'True', 'Predict'])
plt.show()

1. store마다 하나의 모델을 두기 store 마다
2. store마다 family의 모델 두기 